In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [ ]:
#### CALL ####

In [5]:
# Load training and testing data
train_data = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing.xlsx", sheet_name="Training Data")
test_data = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing.xlsx", sheet_name="Testing Data")

In [6]:
# Define the LightGBM parameters
params = {
    'boost_from_average': True,
    'colsample_bytree': 0.5,
    'fair_c': 1,
    'huber_delta': 1,
    'learning_rate': 0.05,
    'max_bin': 255,
    'max_delta_step': 0.7,
    'max_depth': -1,  # None is represented by -1 in LightGBM
    'min_child_samples': 10,
    'min_child_weight': 5,
    'min_split_gain': 0,
    'n_estimators': 2500,
    'num_leaves': 64,
    'objective': 'gamma',
    'reg_alpha': 0,
    'reg_lambda': 0,
    'subsample': 1,
    'subsample_for_bin': 50000,
    'subsample_freq': 1,
    'tweedie_variance_power': 1.5,
    'early_stopping_round': 200  # Set early stopping in parameters
}

# Define lag configurations (individual testing)
lags = {
    'RVlag1': ['RVlag1'],
    'RVlag7': ['RVlag7'],
    'RVlag15': ['RVlag15'],
    'RVlag30': ['RVlag30'],
    'BVlag1': ['BVlag1'],
    'BVlag7': ['BVlag7'],
    'BVlag15': ['BVlag15'],
    'BVlag30': ['BVlag30'],
    'SJVlag7': ['SJVlag7'],
    'SJVlag15': ['SJVlag15'],
    'SJVlag30': ['SJVlag30']
}

In [7]:
# Output file path
output_file = "/Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_SPCall_All.xlsx"

# Create a workbook to save results
wb = Workbook()
ws_summary = wb.active
ws_summary.title = "Summary"
ws_summary.append(["Lag", "RMSE", "MAPE"])  # Add headers for summary

In [8]:
# Loop through each lag configuration
for lag_name, features in lags.items():
    print(f"\nTesting with lag: {lag_name}")
    
    # Select relevant features
    selected_features = ['Price.', 'Maturity', 'Strike'] + features
    
    # Prepare features and target
    x_train = train_data[selected_features].values
    y_train = train_data['Last'].values
    x_test = test_data[selected_features].values
    y_test = test_data['Last'].values
    
    # Create LightGBM datasets
    train_set = lgb.Dataset(x_train, label=y_train)
    test_set = lgb.Dataset(x_test, label=y_test, reference=train_set)
    
    # Train the model
    model = lgb.train(
        params, 
        train_set, 
        valid_sets=[train_set, test_set], 
        valid_names=["train", "test"], 
        num_boost_round=2500
    )
    
    # Make predictions
    y_pred = model.predict(x_test, num_iteration=model.best_iteration)
    
    # Calculate RMSE and MAPE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f"Lag: {lag_name} | RMSE: {rmse:.4f} | MAPE: {mape:.2f}%")
    
    # Save results in a new sheet
    results = pd.DataFrame({
        'Date': test_data['Time'],  # Adjust 'Time' to your date column name
        'Actual': y_test,
        'Predicted': y_pred,
        'RMSE': [rmse] * len(y_test),  # Add RMSE for consistency
        'MAPE': [mape] * len(y_test)  # Add MAPE for consistency
    })
    
    # Add summary results
    ws_summary.append([lag_name, rmse, mape])
    
    # Add sheet for this lag
    ws = wb.create_sheet(title=lag_name)
    for r in dataframe_to_rows(results, index=False, header=True):
        ws.append(r)

# Save the workbook
wb.save(output_file)
print(f"All results saved to Excel file: {output_file}")


Testing with lag: RVlag1
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000386 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 983
[LightGBM] [Info] Number of data points in the train set: 28473, number of used features: 4
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 6.276901
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2497]	train's gamma: 5.66077	test's gamma: 5.69846
Lag: RVlag1 | RMSE: 43.6796 | MAPE: 9.13%

Testing with lag: RVlag7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000089 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 981
[LightGBM] [Info] Number of data points in the train set: 28473, number of used features: 4
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 6.276901
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 5.6607	test's gamma: 5.6981
Lag: RVlag7 | RMSE: 44.0288 | MAPE: 8.96%

Testing with lag: RVlag15
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000227 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 980
[LightGBM] [Info] Number of data points in the train set: 28473, number of used features: 4
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 6.276901
Training until validation scores don't improve for 200 rounds


...

Did not meet early stopping. Best iteration is:
[2497]	train's gamma: 5.66074	test's gamma: 5.69832
Lag: SJVlag7 | RMSE: 43.5880 | MAPE: 9.00%

Testing with lag: SJVlag15
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000088 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 980
[LightGBM] [Info] Number of data points in the train set: 28473, number of used features: 4
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 6.276901
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 5.6606	test's gamma: 5.69799
Lag: SJVlag15 | RMSE: 43.6710 | MAPE: 8.88%

Testing with lag: SJVlag30
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000102 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 981
[LightGBM] [Info] Number of data points in the train set: 28473, number of used features: 4
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 6.276901
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 5.66064	test's gamma: 5.69799
Lag: SJVlag30 | RMSE: 42.6483 | MAPE: 8.82%
All results saved to Excel file: /Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_SPCall_All.xlsx


In [ ]:
### PUT ###

In [9]:
# Read training and testing data for Put options
train_data_put = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing PUT.xlsx", sheet_name="Training Data Put")
test_data_put = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing PUT.xlsx", sheet_name="Testing Data Put")

In [10]:
# Define the LightGBM parameters
params = {
    'boost_from_average': True,
    'colsample_bytree': 0.5,
    'fair_c': 1,
    'huber_delta': 1,
    'learning_rate': 0.05,
    'max_bin': 255,
    'max_delta_step': 0.7,
    'max_depth': -1,  # None is represented by -1 in LightGBM
    'min_child_samples': 10,
    'min_child_weight': 5,
    'min_split_gain': 0,
    'n_estimators': 2500,
    'num_leaves': 16,
    'objective': 'gamma',
    'reg_alpha': 0,
    'reg_lambda': 0,
    'subsample': 1,
    'subsample_for_bin': 50000,
    'subsample_freq': 1,
    'tweedie_variance_power': 1.5,
    'early_stopping_round': 200  # Set early stopping in parameters
}

# Define lag configurations (individual testing)
lags = {
    'RVlag1': ['RVlag1'],
    'RVlag7': ['RVlag7'],
    'RVlag15': ['RVlag15'],
    'RVlag30': ['RVlag30'],
    'BVlag1': ['BVlag1'],
    'BVlag7': ['BVlag7'],
    'BVlag15': ['BVlag15'],
    'BVlag30': ['BVlag30'],
    'SJVlag7': ['SJVlag7'],
    'SJVlag15': ['SJVlag15'],
    'SJVlag30': ['SJVlag30']
}

In [11]:
# Output file path
output_file = "/Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_SPPut_All.xlsx"

# Create a workbook to save results
wb = Workbook()
ws_summary = wb.active
ws_summary.title = "Summary"
ws_summary.append(["Lag", "RMSE", "MAPE"])  # Add headers for summary

In [12]:
# Loop through each lag configuration
for lag_name, features in lags.items():
    print(f"\nTesting with lag: {lag_name}")
    
    # Select relevant features
    selected_features = ['Price.', 'Maturity', 'Strike'] + features
    
    # Prepare features and target
    x_train = train_data_put[selected_features].values
    y_train = train_data_put['Last'].values
    x_test = test_data_put[selected_features].values
    y_test = test_data_put['Last'].values
    
    # Create LightGBM datasets
    train_set = lgb.Dataset(x_train, label=y_train)
    test_set = lgb.Dataset(x_test, label=y_test, reference=train_set)
    
    # Train the model
    model = lgb.train(
        params, 
        train_set, 
        valid_sets=[train_set, test_set], 
        valid_names=["train", "test"], 
        num_boost_round=2500
    )
    
    # Make predictions
    y_pred = model.predict(x_test, num_iteration=model.best_iteration)
    
    # Calculate RMSE and MAPE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f"Lag: {lag_name} | RMSE: {rmse:.4f} | MAPE: {mape:.2f}%")
    
    # Save results in a new sheet
    results = pd.DataFrame({
        'Date': test_data_put['Time'],  # Adjust 'Time' to your date column name
        'Actual': y_test,
        'Predicted': y_pred,
        'RMSE': [rmse] * len(y_test),  # Add RMSE for consistency
        'MAPE': [mape] * len(y_test)  # Add MAPE for consistency
    })
    
    # Add summary results
    ws_summary.append([lag_name, rmse, mape])
    
    # Add sheet for this lag
    ws = wb.create_sheet(title=lag_name)
    for r in dataframe_to_rows(results, index=False, header=True):
        ws.append(r)

# Save the workbook
wb.save(output_file)
print(f"All results saved to Excel file: {output_file}")


Testing with lag: RVlag1
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000471 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 988
[LightGBM] [Info] Number of data points in the train set: 38394, number of used features: 4
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 4.684843
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 3.98478	test's gamma: 3.97509
Lag: RVlag1 | RMSE: 26.1962 | MAPE: 12.01%

Testing with lag: RVlag7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000164 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 990
[LightGBM] [Info] Number of data points in the train set: 38394, number of used features: 4
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 4.684843
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 3.98469	test's gamma: 3.97512
Lag: RVlag7 | RMSE: 28.0416 | MAPE: 11.95%

Testing with lag: RVlag15
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000268 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 989
[LightGBM] [Info] Number of data points in the train set: 38394, number of used features: 4
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 4.684843
Training until validation scores don't improve for 200 rounds


...

Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 3.98452	test's gamma: 3.97509
Lag: SJVlag7 | RMSE: 27.5317 | MAPE: 11.93%

Testing with lag: SJVlag15
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000134 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 985
[LightGBM] [Info] Number of data points in the train set: 38394, number of used features: 4
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 4.684843
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2499]	train's gamma: 3.9846	test's gamma: 3.97496
Lag: SJVlag15 | RMSE: 25.6245 | MAPE: 11.95%

Testing with lag: SJVlag30
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000285 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 985
[LightGBM] [Info] Number of data points in the train set: 38394, number of used features: 4
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 4.684843
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 3.98458	test's gamma: 3.97492
Lag: SJVlag30 | RMSE: 25.9610 | MAPE: 11.95%
All results saved to Excel file: /Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_SPPut_All.xlsx
